In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import re

# Question 1

## Import Cities and NHL Dataset 

In [2]:
nhl_df = pd.read_csv('assets/nhl.csv')

In [3]:
#nhl_df.head()

In [4]:
nhl_df = nhl_df[nhl_df['year'] == 2018]

In [104]:
#nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,team_LName
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL,Division
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Lightning
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Bruins
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Leafs
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Panthers


In [6]:
cities_df = pd.read_html('assets/wikipedia_data.html')[1]

In [7]:
#cities_df.head()

In [8]:
cities_df.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)

In [9]:
new_nhl = cities_df[['Metropolitan area', 'NHL']]

In [10]:
new_nhl['LName'] = new_nhl['NHL'].apply(lambda x: re.findall('[A-Z][a-z]+', x))

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
new_nhl = new_nhl.explode(column='LName').drop('NHL', axis=1).dropna()

In [12]:
new_nhl.rename({'LName': 'team_LName'}, axis=1, inplace=True)

In [13]:
#new_nhl.head()

In [14]:
nhlteams_index = new_nhl['Metropolitan area'].unique()

In [15]:
nhl_df['team_LName'] = nhl_df['team'].apply(lambda x: re.findall('.+\s(\w+)\*?', x)[-1])

In [16]:
#nhl_df.head()

In [17]:
nhl_cities = nhl_df.merge(new_nhl, right_on='team_LName', left_on='team_LName')

In [18]:
#nhl_cities.head()

In [19]:
nhl_cities['W'] = nhl_cities['W'].astype(int)
nhl_cities['L'] = nhl_cities['L'].astype(int)

In [20]:
nhl_cities['W/L_%'] = nhl_cities['W'] / (nhl_cities['L'] + nhl_cities['W'])

In [21]:
#nhl_cities.head()

In [22]:
nhl_cities_merged = nhl_cities.groupby('Metropolitan area').agg({'W/L_%': 'mean'}).sort_index()

In [23]:
#cities_df.head()

In [24]:
q1 = nhl_cities_merged.merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area')

In [25]:
q1 = q1.set_index('Metropolitan area').loc[nhlteams_index]

In [26]:
q1.head()

,W/L_%,Population
Metropolitan area,,
New York City,0.518201,20153634
Los Angeles,0.622895,13310447
San Francisco Bay Area,0.625000,6657982
Chicago,0.458333,9512999
Dallas–Fort Worth,0.567568,7233323


In [27]:
#cities_df=cities_df.iloc[:-1,[0,3,5,6,7,8]]

In [28]:
#cities_df

In [29]:
q1['Population'] = q1['Population'].astype(np.float)

In [30]:
stats.pearsonr(q1['W/L_%'], q1['Population'])

(0.012486162921209909, 0.9497182859911791)


# Question 2

## Import NBA Dataset

In [31]:
nba_df = pd.read_csv('assets/nba.csv')

In [32]:
nba_df = nba_df[nba_df['year'] == 2018]

In [33]:
#nba_df.head()

In [34]:
#cities_df.head()

In [35]:
#re.findall('[A-Z\d]+[a-z]+', cities_df['NBA'][6])

In [36]:
new_nba = cities_df[['Metropolitan area', 'NBA']]
new_nba['team_LName'] = cities_df['NBA'].apply(lambda x: re.findall('[A-Z\d]+[a-z]+', x))

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
#new_nba

In [38]:
new_nba = new_nba.explode(column='team_LName').dropna().drop('NBA', axis=1)

In [39]:
#new_nba.shape

In [40]:
#nba_df.head()

In [41]:
#re.findall('[a-zA-Z\s]+\s([1-9a-zA-Z]+)\*?\s\S+', nba_df['team'][2])[0]

In [42]:
#nba_df['team'][]

In [43]:
nba_df['team_LName'] = nba_df['team'].apply(lambda x: re.findall('[a-zA-Z\s]+\s([1-9a-zA-Z]+)\*?\s\S+', x)[0])

In [44]:
#nba_df.head()

In [45]:
nba_df['W/L%'] = nba_df['W/L%'].astype(np.float)

In [46]:
#nba_df.merge(new_nba, on='team_LName')

In [47]:
nba_cities_merged = new_nba.merge(nba_df, on='team_LName').groupby('Metropolitan area').agg({'W/L%': 'mean'}).reset_index()

In [48]:
#nba_cities_merged

In [49]:
q2 = cities_df.merge(nba_cities_merged, on='Metropolitan area')[['Metropolitan area', 'W/L%', 'Population']].set_index('Metropolitan area')

In [50]:
q2['Population'] = q2['Population'].astype(np.float)

In [51]:
stats.pearsonr(q2['Population'], q2['W/L%'])

(-0.17636350642182935, 0.36932106185547353)

# Question 3

## Import MLB Dataset

In [52]:
mlb_df = pd.read_csv('assets/mlb.csv')

In [53]:
mlb_df = mlb_df[mlb_df['year'] == 2018]

In [54]:
#mlb_df.shape

In [55]:
#cities_df.head()

In [56]:
new_mlb = cities_df[['Metropolitan area', 'MLB']]

In [57]:
new_mlb.loc[:, 'team_LName'] = cities_df.loc[:, 'MLB'].apply(lambda x: re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x))

C:\Users\Mann\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Mann\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [58]:
new_mlb = new_mlb.explode(column='team_LName').dropna().drop('MLB', axis=1)

In [59]:
mlb_index = new_mlb['Metropolitan area'].unique()

In [60]:
#mlb_df.head()

In [61]:
mlb_df.loc[:, 'team_LName'] = mlb_df.loc[:, 'team'].apply(lambda x: re.findall('.+\s([A-Z][a-z]+)', x)[0])

In [62]:
mlb_df.loc[0, 'team_LName'] = 'Red Sox'
mlb_df.loc[8, 'team_LName'] = 'White Sox'
mlb_df.loc[3, 'team_LName'] = 'Blue Jays'

In [63]:
#mlb_df

In [64]:
#new_mlb

In [65]:
q3 = new_mlb.merge(mlb_df, on='team_LName').groupby('Metropolitan area').agg({'W-L%': 'mean'}).loc[mlb_index, :]

In [66]:
q3 = q3.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')

In [67]:
q3['W-L%'] = q3['W-L%'].astype(np.float)
q3['Population'] = q3['Population'].astype(np.float)

In [68]:
#q3

In [69]:
stats.pearsonr(q3['W-L%'], q3['Population'])

(0.15003737475409495, 0.46442827201123427)

# Question 4

## Import NFL Dataset

In [70]:
nfl_df = pd.read_csv('assets/nfl.csv')

In [71]:
nfl_df = nfl_df[nfl_df['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35], axis=0)

In [72]:
#nfl_df.head()

In [73]:
nfl_df['W-L%'] = nfl_df['W-L%'].astype(np.float)

In [74]:
#cities_df.head()

In [75]:
new_nfl = cities_df[['Metropolitan area', 'NFL']]

In [76]:
new_nfl['team_LName'] = cities_df['NFL'].apply(lambda x: re.findall('[A-Z\d]+[a-z]+', x))

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
new_nfl = new_nfl.explode(column='team_LName').dropna().drop('NFL', axis=1)

In [78]:
#new_nfl.head()

In [79]:
#nfl_df

In [80]:
nfl_df['team_LName'] = nfl_df['team'].apply(lambda x: re.findall('.+\s([A-Z1-9]+[a-z]+)\S*', x)[0])

In [81]:
#nfl_df.head()

In [82]:
q4 = nfl_df.merge(new_nfl, on='team_LName').groupby('Metropolitan area').agg({'W-L%': 'mean'})

In [83]:
q4 = q4.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')

In [84]:
#q4.dtypes

In [85]:
q4['Population'] = q4['Population'].astype(np.float)

In [86]:
stats.pearsonr(q4['Population'], q4['W-L%'])

(0.004282141436393022, 0.9824114740736553)

# Question 5

In [87]:
q1.head()

,W/L_%,Population
Metropolitan area,,
New York City,0.518201,20153634.0
Los Angeles,0.622895,13310447.0
San Francisco Bay Area,0.625000,6657982.0
Chicago,0.458333,9512999.0
Dallas–Fort Worth,0.567568,7233323.0


In [88]:
q2.head()

,W/L%,Population
Metropolitan area,,
New York City,0.3475,20153634.0
Los Angeles,0.4695,13310447.0
San Francisco Bay Area,0.7070,6657982.0
Chicago,0.3290,9512999.0
Dallas–Fort Worth,0.2930,7233323.0


In [89]:
q3.head()

,W-L%,Population
Metropolitan area,,
New York City,0.546,20153634.0
Los Angeles,0.529,13310447.0
San Francisco Bay Area,0.525,6657982.0
Chicago,0.483,9512999.0
Dallas–Fort Worth,0.414,7233323.0


In [90]:
q4.head()

,W-L%,Population
Metropolitan area,,
Atlanta,0.438,5789700.0
Baltimore,0.625,2798886.0
Boston,0.688,4794447.0
Buffalo,0.375,1132804.0
Charlotte,0.438,2474314.0


In [93]:
q1.reset_index().merge(q2.reset_index(), on='Metropolitan area').set_index('Metropolitan area')

,W/L_%,Population_x,W/L%,Population_y
Metropolitan area,,,,
New York City,0.518201,20153634.0,0.3475,20153634.0
Los Angeles,0.622895,13310447.0,0.4695,13310447.0
San Francisco Bay Area,0.625000,6657982.0,0.7070,6657982.0
Chicago,0.458333,9512999.0,0.3290,9512999.0
Dallas–Fort Worth,0.567568,7233323.0,0.2930,7233323.0
"Washington, D.C.",0.653333,6131977.0,0.5240,6131977.0
Philadelphia,0.617647,6070500.0,0.6340,6070500.0
Boston,0.714286,4794447.0,0.6710,4794447.0
Minneapolis–Saint Paul,0.633803,3551036.0,0.5730,3551036.0


In [94]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

In [95]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,NaN,NaN,NaN
NBA,NaN,NaN,NaN,NaN
NHL,NaN,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN


In [96]:
q1_nhl = q1
q2_nba = q2
q3_mlb = q3
q4_nfl = q4

In [103]:
q1_nhl.merge(q2_nba)

,W/L_%,Population,W/L%
0,0.518201,20153634.0,0.3475
1,0.622895,13310447.0,0.4695
2,0.625000,6657982.0,0.7070
3,0.458333,9512999.0,0.3290
4,0.567568,7233323.0,0.2930
5,0.653333,6131977.0,0.5240
6,0.617647,6070500.0,0.6340
7,0.714286,4794447.0,0.6710
8,0.633803,3551036.0,0.5730
9,0.589041,2853077.0,0.5610


In [113]:
p_values.loc['NHL', 'NBA'] = stats.ttest_rel(q1_nhl.merge(q2_nba).iloc[:, 0], q1_nhl.merge(q2_nba).iloc[:, 2])[1]
p_values.loc['NBA', 'NHL'] = p_values.loc['NHL', 'NBA']

In [120]:
q3_mlb

,W-L%,Population
Metropolitan area,,
New York City,0.546,20153634.0
Los Angeles,0.529,13310447.0
San Francisco Bay Area,0.525,6657982.0
Chicago,0.483,9512999.0
Dallas–Fort Worth,0.414,7233323.0
"Washington, D.C.",0.506,6131977.0
Philadelphia,0.494,6070500.0
Boston,0.667,4794447.0
Minneapolis–Saint Paul,0.481,3551036.0


In [130]:
mlb_nfl = q3_mlb.reset_index().merge(q4_nfl.reset_index(), on='Metropolitan area').iloc[:, [1, 3]]

In [136]:
p_values.loc['MLB', 'NFL'] = stats.ttest_rel(mlb_nfl.iloc[:, 0], mlb_nfl.iloc[:, 1])[1]
p_values.loc['NFL', 'MLB'] = p_values.loc['MLB', 'NFL']

In [137]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,NaN,NaN,0.803459
NBA,NaN,NaN,0.022386,NaN
NHL,NaN,0.022386,NaN,NaN
MLB,0.803459,NaN,NaN,NaN


In [139]:
nfl_nba = q2_nba.merge(q4_nfl)

In [157]:
p_values.loc['NBA', 'NFL'] = stats.ttest_rel(nfl_nba.iloc[:, 0], nfl_nba.iloc[:, 2])[1]
p_values.loc['NFL', 'NBA'] = p_values.loc['NBA', 'NFL']

In [160]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,NaN
NHL,0.030318,0.022386,NaN,NaN
MLB,0.803459,NaN,NaN,NaN


In [149]:
nfl_nhl = q1_nhl.merge(q4_nfl)

In [161]:
nfl_nhl

,W/L_%,Population,W-L%
0,0.518201,20153634.0,0.2815
1,0.622895,13310447.0,0.7815
2,0.625000,6657982.0,0.2500
3,0.458333,9512999.0,0.7500
4,0.567568,7233323.0,0.6250
5,0.653333,6131977.0,0.4380
6,0.617647,6070500.0,0.5630
7,0.714286,4794447.0,0.6880
8,0.633803,3551036.0,0.5310
9,0.589041,2853077.0,0.3750


In [159]:
p_values.loc['NFL', 'NHL'] = stats.ttest_rel(nfl_nhl.iloc[:, 0], nfl_nhl.iloc[:, 2])[1]
p_values.loc['NHL', 'NFL'] = p_values.loc['NFL', 'NHL']

In [162]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,NaN
NHL,0.030318,0.022386,NaN,NaN
MLB,0.803459,NaN,NaN,NaN


In [155]:
nba_mlb = q2_nba.merge(q3_mlb)

In [165]:
p_values.loc['NBA', 'MLB'] = stats.ttest_rel(nba_mlb.iloc[:, 0], nba_mlb.iloc[:, 2])[1]
p_values.loc['MLB', 'NBA'] = p_values.loc['NBA', 'MLB']

In [166]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,NaN
MLB,0.803459,0.949566,NaN,NaN


In [168]:
nhl_mlb = q1_nhl.merge(q3_mlb)

In [172]:
p_values.loc['NHL', 'MLB'] = stats.ttest_rel(nhl_mlb.iloc[:, 0], nhl_mlb.iloc[:, 2])[1]
p_values.loc['MLB', 'NHL'] = p_values.loc['NHL', 'MLB']

In [173]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
